In [ ]:
#content_filtering code

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load dataset
data = pd.read_excel("sample_data.xlsx")

# Preprocessing
# Encode categorical variables
label_encoders = {}
for col in ["state", "category"]:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Split data
X = data.drop(["place1", "place2", "place3"], axis=1)
y = data[["place1", "place2", "place3"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train model (example using RandomForestClassifier)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# User input (state, category, rating)
user_input_state = "Uttarakhand"  # Replace with actual user input
user_input_category = "Religious"
user_input_rating = 4.5

# Process user input
user_input_encoded = pd.DataFrame({
    "state": [label_encoders["state"].transform([user_input_state])[0]],
    "category": [label_encoders["category"].transform([user_input_category])[0]],
    "rating": [user_input_rating]
})

# Get recommendations
predictions = model.predict(user_input_encoded)

print("Top 3 recommended places:", predictions)


Top 3 recommended places: [['Haridwar' ' Panch Kedar' ' Hemkund Sahib']]


In [ ]:
#Content_filtering_using_tensorflow

In [38]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import tensorflow as tf
import numpy as np

# Load the data
data = pd.read_excel('sample_data.xlsx')

# Initialize label encoders for categorical features and outputs
label_encoders = {}
for column in ['state', 'category', 'place1', 'place2', 'place3']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Normalize the rating column
scaler = MinMaxScaler()
data['rating'] = scaler.fit_transform(data[['rating']])

# Separate features and outputs
X = data[['state', 'category', 'rating']].values
y1 = data['place1'].values
y2 = data['place2'].values
y3 = data['place3'].values

# Convert outputs to one-hot encoding
y1 = tf.keras.utils.to_categorical(y1, num_classes=len(label_encoders['place1'].classes_))
y2 = tf.keras.utils.to_categorical(y2, num_classes=len(label_encoders['place2'].classes_))
y3 = tf.keras.utils.to_categorical(y3, num_classes=len(label_encoders['place3'].classes_))

# Input shape
input_shape = X.shape[1]

# Define the model
input_layer = tf.keras.layers.Input(shape=(input_shape,))
dense_layer = tf.keras.layers.Dense(64, activation='relu')(input_layer)
dense_layer = tf.keras.layers.Dense(64, activation='relu')(dense_layer)

# Output layers for place1, place2, and place3
output1 = tf.keras.layers.Dense(len(label_encoders['place1'].classes_), activation='softmax')(dense_layer)
output2 = tf.keras.layers.Dense(len(label_encoders['place2'].classes_), activation='softmax')(dense_layer)
output3 = tf.keras.layers.Dense(len(label_encoders['place3'].classes_), activation='softmax')(dense_layer)

# Create the model
model = tf.keras.models.Model(inputs=input_layer, outputs=[output1, output2, output3])

# Compile the model with separate metrics for each output
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy', 'accuracy', 'accuracy'])

# Train the model
model.fit(X, [y1, y2, y3], epochs=150, batch_size=32, validation_split=0.2)

# Function to preprocess user input
def preprocess_input(state, category, rating):
    state_encoded = label_encoders['state'].transform([state])
    category_encoded = label_encoders['category'].transform([category])
    rating_normalized = scaler.transform([[rating]])
    return np.array([state_encoded[0], category_encoded[0], rating_normalized[0][0]])

# Function to predict and decode recommendations
def recommend_places(state, category, rating):
    input_data = preprocess_input(state, category, rating)
    input_data = np.expand_dims(input_data, axis=0)
    predictions = model.predict(input_data)
    place1 = label_encoders['place1'].inverse_transform([np.argmax(predictions[0])])[0]
    place2 = label_encoders['place2'].inverse_transform([np.argmax(predictions[1])])[0]
    place3 = label_encoders['place3'].inverse_transform([np.argmax(predictions[2])])[0]
    return place1, place2, place3

# Example usage
state = 'Uttarakhand'
category = 'Hill Station'
rating = 4.0
recommendations = recommend_places(state, category, rating)
print(f'Recommended places: {recommendations}')



Epoch 1/150
44/44 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - dense_77_accuracy: 0.0096 - dense_78_accuracy: 0.0298 - dense_79_accuracy: 0.0256 - loss: 13.4359 - val_dense_77_accuracy: 0.1136 - val_dense_78_accuracy: 0.0767 - val_dense_79_accuracy: 0.1364 - val_loss: 12.5596
Epoch 2/150
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - dense_77_accuracy: 0.0978 - dense_78_accuracy: 0.0672 - dense_79_accuracy: 0.1109 - loss: 12.4705 - val_dense_77_accuracy: 0.1420 - val_dense_78_accuracy: 0.0938 - val_dense_79_accuracy: 0.1619 - val_loss: 11.2408
Epoch 3/150
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - dense_77_accuracy: 0.0975 - dense_78_accuracy: 0.0662 - dense_79_accuracy: 0.1213 - loss: 11.4205 - val_dense_77_accuracy: 0.1875 - val_dense_78_accuracy: 0.1335 - val_dense_79_accuracy: 0.1932 - val_loss: 9.8574
Epoch 4/150
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - dense_77_accuracy: 0.1649 - dense_78_accuracy: 0.0892 - dense_79_accuracy: 0.1648 - loss: 10.1836 - val_dense_77_accuracy: 0.2585 - val_dense_78_a

C:\Users\h8888\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
Recommended places: ('Haldwani', ' Pauri', ' Bhowali')


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import tensorflow as tf
import numpy as np

# Load the data
data = pd.read_excel('sample_data.xlsx')

# Initialize label encoders for categorical features and outputs
label_encoders = {}
for column in ['state', 'category', 'place1', 'place2', 'place3']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Normalize the rating column
scaler = MinMaxScaler()
data['rating'] = scaler.fit_transform(data[['rating']])

# Separate features and outputs
X = data[['state', 'category', 'rating']].values
y1 = data['place1'].values
y2 = data['place2'].values
y3 = data['place3'].values

# Convert outputs to one-hot encoding
y1 = tf.keras.utils.to_categorical(y1, num_classes=len(label_encoders['place1'].classes_))
y2 = tf.keras.utils.to_categorical(y2, num_classes=len(label_encoders['place2'].classes_))
y3 = tf.keras.utils.to_categorical(y3, num_classes=len(label_encoders['place3'].classes_))

# Input shape
input_shape = X.shape[1]

# Define the model
input_layer = tf.keras.layers.Input(shape=(input_shape,))
dense_layer1 = tf.keras.layers.Dense(64, activation='relu')(input_layer)
dense_layer2 = tf.keras.layers.Dense(64, activation='relu')(dense_layer1)
output_layer1 = tf.keras.layers.Dense(len(label_encoders['place1'].classes_), activation='softmax')(dense_layer2)
output_layer2 = tf.keras.layers.Dense(len(label_encoders['place2'].classes_), activation='softmax')(dense_layer2)
output_layer3 = tf.keras.layers.Dense(len(label_encoders['place3'].classes_), activation='softmax')(dense_layer2)

# Create the model
model = tf.keras.models.Model(inputs=input_layer, outputs=[output_layer1, output_layer2, output_layer3])

# Compile the model with separate metrics for each output
model.compile(optimizer='adam', 
              loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'], 
              metrics=['accuracy', 'accuracy', 'accuracy'])

# Train the model
model.fit(X, [y1, y2, y3], epochs=150, batch_size=32, validation_split=0.2)

# Save the model as an HDF5 file
model.save('saved_model.h5')

# Load the model for conversion
loaded_model = tf.keras.models.load_model('saved_model.h5')

# Convert the model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
tflite_model = converter.convert()

# Save the TFLite model to a file
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

# Function to preprocess user input
def preprocess_input(state, category, rating):
    state_encoded = label_encoders['state'].transform([state])
    category_encoded = label_encoders['category'].transform([category])
    rating_normalized = scaler.transform([[rating]])
    return np.array([state_encoded[0], category_encoded[0], rating_normalized[0][0]])

# Function to predict and decode recommendations
def recommend_places(state, category, rating):
    input_data = preprocess_input(state, category, rating)
    input_data = np.expand_dims(input_data, axis=0)
    predictions = loaded_model.predict(input_data)
    place1 = label_encoders['place1'].inverse_transform([np.argmax(predictions[0])])[0]
    place2 = label_encoders['place2'].inverse_transform([np.argmax(predictions[1])])[0]
    place3 = label_encoders['place3'].inverse_transform([np.argmax(predictions[2])])[0]
    return place1, place2, place3

# Example usage
state = 'Delhi'
category = 'Natural Beauty'
rating = 4.0
recommendations = recommend_places(state, category, rating)
print(f'Recommended places: {recommendations}')


Epoch 1/150
44/44 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - dense_17_accuracy: 0.0549 - dense_18_accuracy: 0.0148 - dense_19_accuracy: 0.0250 - loss: 13.4093 - val_dense_17_accuracy: 0.1136 - val_dense_18_accuracy: 0.0540 - val_dense_19_accuracy: 0.0795 - val_loss: 12.4447
Epoch 2/150
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - dense_17_accuracy: 0.0923 - dense_18_accuracy: 0.0440 - dense_19_accuracy: 0.0832 - loss: 12.3810 - val_dense_17_accuracy: 0.1875 - val_dense_18_accuracy: 0.0994 - val_dense_19_accuracy: 0.1619 - val_loss: 11.2733
Epoch 3/150
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - dense_17_accuracy: 0.1773 - dense_18_accuracy: 0.0859 - dense_19_accuracy: 0.1434 - loss: 11.2557 - val_dense_17_accuracy: 0.2131 - val_dense_18_accuracy: 0.1278 - val_dense_19_accuracy: 0.1903 - val_loss: 9.8804
Epoch 4/150
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - dense_17_accuracy: 0.1973 - dense_18_accuracy: 0.0896 - dense_19_accuracy: 0.1597 - loss: 10.1161 - val_dense_17_accuracy: 0.2131 - val_dense_18_a